<a href="https://colab.research.google.com/github/aknip/llm-benchmark/blob/main/llm_simple_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tiktoken litellm pandas openpyxl bs4 icecream --quiet

In [2]:
import json
import os
from getpass import getpass
import psutil
import requests
import textwrap
from icecream import ic
import time
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [3]:
import openai
from litellm import completion
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential'] # my key
os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']

In [5]:
import time

In [4]:
import openai
import os

client = openai.OpenAI(
  api_key=os.environ.get("TOGETHERAI_API_KEY"),
  base_url='https://api.together.xyz',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you with any language-related questions you have. Is there something specific you would like to know or practice?


# Call via litellm

In [5]:
response = completion(
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with any questions you have. Is there something specific you would like to know or talk about?
ModelResponse(id='chatcmpl-80ba4d38-ee9f-42f1-b2e4-71c4db0f487e', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with any questions you have. Is there something specific you would like to know or talk about?", role='assistant'))], created=1703926174, model='together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=12, completion_tokens=44, total_tokens=56), _response_ms=2074.8419999999996)


In [6]:
response = completion(
  model="gpt-3.5-turbo",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

I'm an AI, so I don't have feelings, but I'm here to assist you. How can I help you today?
ModelResponse(id='chatcmpl-8bPKKlY5fJEniH6li1bSHil8wYqAh', choices=[Choices(finish_reason='stop', index=0, message=Message(content="I'm an AI, so I don't have feelings, but I'm here to assist you. How can I help you today?", role='assistant'))], created=1703926183, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=Usage(completion_tokens=27, prompt_tokens=13, total_tokens=40), _response_ms=1607.078)


# Function Calling

In [ ]:
# see https://litellm.vercel.app/docs/completion/function_call

# via Huggingface?
# https://litellm.vercel.app/docs/providers/huggingface
# https://huggingface.co/Trelis/Mixtral-8x7B-Instruct-v0.1-function-calling-v3
# https://huggingface.co/Trelis/Mistral-7B-Instruct-v0.1-function-calling-v2

# via Anyscale?
# https://docs.litellm.ai/docs/providers/anyscale
# https://www.anyscale.com/blog/anyscale-endpoints-json-mode-and-function-calling-features

In [7]:
import os, litellm
from litellm import completion

# IMPORTANT - Set this to TRUE to add the function to the prompt for Non OpenAI LLMs
litellm.add_function_to_prompt = True

# The real function is not needed for the LLM. It may be called after the LLM call (not in this code!)
def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

response = completion(model="gpt-3.5-turbo-1106", messages=messages, functions=functions)

print(response)
print()
function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_name)
else:
  print('No function found')

ModelResponse(id='chatcmpl-8bPKfEySA6R1vl8rdfdB9cFJdphsh', choices=[Choices(finish_reason='function_call', index=0, message=Message(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather')))], created=1703926205, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=Usage(completion_tokens=17, prompt_tokens=82, total_tokens=99), _response_ms=998.558)

get_current_weather


# 1. Create Excel file

In [8]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# see https://openpyxl.readthedocs.io/en/2.4/pandas.html

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Name': ['Miller', 'Adams', 'Smith'],
                   'Prompt': ['Write something', 'Do different things', 'Summarize it'], })

# Create a Pandas Excel writer using openpyxl as the engine.
writer = pd.ExcelWriter('llm_benchmark.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Benchmark', index=True)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Benchmark']

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

# Alternate approach without ExcelWriter:
# workbook = Workbook()
# worksheet = workbook.active
# for row in dataframe_to_rows(df, index=True, header=True):
#     if row != [None]:
#       worksheet.append(row)

# 2. Read Excel file as dataframe

In [9]:
import pandas as pd
#import xlrd

df = pd.read_excel('llm_benchmark.xlsx', sheet_name='Benchmark') # parse_dates=['date'] # dtype={'column_name': float}
#print(df.head())

print("Given Dataframe :\n", df)

print("\nIterating 1:")
for i in df.index:
    print(i, df['Name'][i],df['Prompt'][i])

print("\nIterating 2:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")

Given Dataframe :
    Unnamed: 0    Name               Prompt
0           0  Miller      Write something
1           1   Adams  Do different things
2           2   Smith         Summarize it

Iterating 1:
0 Miller Write something
1 Adams Do different things
2 Smith Summarize it

Iterating 2:
0: Miller, Write something
1: Adams, Do different things
2: Smith, Summarize it


# 3. Updata data and Excel file

In [10]:
df.at[1, 'Name'] = 'Name-Updated'
df.at[2, 'Prompt'] = 'Prompt-Updated'
print(df.head())

   Unnamed: 0          Name               Prompt
0           0        Miller      Write something
1           1  Name-Updated  Do different things
2           2         Smith       Prompt-Updated


In [11]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
workbook = load_workbook(filename = 'llm_benchmark.xlsx')
worksheet = workbook.active

print("Updating Excel file:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")
    d = worksheet.cell(row=index+2, column=2, value=name)
    d = worksheet.cell(row=index+2, column=3, value=prompt)

# Alternative: delete data rows and append dataframe to worksheet
# continuously delete row 2 until there is only first row (header row) is left over
# while(worksheet.max_row > 1):
#     worksheet.delete_rows(2) # removes the row 2
# append dataframe to worksheet
# for row in dataframe_to_rows(df, index=False, header=False):
#     if row != [None]:
#        worksheet.append(row)

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

Updating Excel file:
0: Miller, Write something
1: Name-Updated, Do different things
2: Smith, Prompt-Updated


# Create file, read file, save to Excel

In [39]:
email_text = '''Enter some text here
and here
and here
'''

In [40]:
f= open('24-anonymized-shortened.txt','w+')
f.write(email_text)
f.close()

In [49]:
import tiktoken

f= open('24-anonymized-shortened.txt','r')
if f.mode == 'r':
      email_text =f.read()
      #print(email_text)
f.close()

email_text_tokens = len(tiktoken.get_encoding('cl100k_base').encode(email_text))

In [50]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
workbook = load_workbook(filename = 'llm_benchmark.xlsx')
worksheet = workbook.active

print("Writing email-text to Excel file...")
worksheet.cell(row=5, column=1, value=3)
worksheet.cell(row=5, column=2, value="mytest")
worksheet.cell(row=5, column=3, value=email_text)
worksheet.cell(row=5, column=4, value=email_text_tokens)
workbook.save('llm_benchmark.xlsx')

Writing email-text to Excel file...


# Read email from Excel

In [161]:
import pandas as pd

df = pd.read_excel('llm_benchmark.xlsx', sheet_name='Benchmark') # parse_dates=['date'] # dtype={'column_name': float}
email_text = df['Prompt'][3]
print(df.head())
#print(df['Name'][3],df['Prompt'][3][:500])

   Unnamed: 0    Name                                             Prompt  \
0           0  Miller                                    Write something   
1           1   Adams                                Do different things   
2           2   Smith                                       Summarize it   
3           3  mytest  # Email Nachricht:\n\nFrom: Ralf Löffler - Uni...   

   Unnamed: 3  
0         NaN  
1         NaN  
2         NaN  
3      2677.0  


In [53]:
import tiktoken
email_text_tokens = len(tiktoken.get_encoding('cl100k_base').encode(email_text))
print(email_text_tokens)


2677


In [54]:
prompt1 = '''Nachfolgend sende ich alle Informationen für eine Anfrage eines Versicherungsmaklers für eine D&O-Versicherung seines Kunden. Die Informationen bestehen aus:
- Email Nachricht
- Organigramm
- Versicherungsschein
- Nachtrag
- Versicherungsbedingungen
'''

In [56]:
prompt2 = 'Bitte bestätige mit "Ja" wenn Du die Inhalte verstanden hast.'

In [103]:
prompt3 = 'Wie hoch ist die Versicherungssumme? Antworte mit folgendem JSON-Schema: {"Versicherungssumme": ""}, formatiere Zahlen in europäischer Notation.'

In [87]:
full_prompt = prompt1 + '\n\n' + prompt3 + '\n\n' + email_text

In [153]:
response = completion(
  #model="together_ai/togethercomputer/llama-2-70b",
  #model="together_ai/togethercomputer/llama-2-70b-chat",
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  #model="gpt-3.5-turbo",
  messages=[{ "content": full_prompt,"role": "user"}],
  #temperature=0.5,
  max_tokens=100
)

print(response)
print(response.choices[0].message.content)

ModelResponse(id='chatcmpl-a14a4efb-9586-46c2-a7b8-6f9b87b1a9df', choices=[Choices(finish_reason='stop', index=0, message=Message(content='{\n"Versicherungssumme": "5.500.000,00 €"\n}', role='assistant'))], created=1703938939, model='together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=2779, completion_tokens=20, total_tokens=2799), _response_ms=2487.099)
{
"Versicherungssumme": "5.500.000,00 €"
}


In [ ]:
from bs4 import BeautifulSoup
response = requests.get("https://www.tagesschau.de/inland/hochwasser-deutschland-126.html")
soup = BeautifulSoup(response.text, 'html.parser')
all_text = soup.get_text(separator='\n', strip=True)
div_text=soup.find("div",{"class":"layout-container"}).get_text()
article_text=soup.find("article").get_text(separator='\n', strip=True)
print(article_text)

In [160]:
response = completion(
  #model="together_ai/togethercomputer/llama-2-70b",
  #model="together_ai/togethercomputer/llama-2-70b-chat",
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  #model="gpt-3.5-turbo",
  messages=[{ "content": "Summarize the following text in 3 sentences, answer in German language: " + article_text,"role": "user"}],
  #temperature=0.5,
  max_tokens=500
)

print(response)
print(textwrap.fill(response.choices[0].message.content, 80))

ModelResponse(id='chatcmpl-256bdced-aaf7-4b16-ba09-76d0b1c71d8d', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Die Hochwasserlage in Niedersachsen ist weiter kritisch, insbesondere in den Landkreisen Celle, Heidekreis, Verden, Emsland und Oldenburg. Obwohl sich die Lage in einigen Gebieten entspannt hat, steigen die Pegelstände an anderen Orten noch. Das THW ist auf einen Einsatz in den Hochwassergebieten bis in die erste Januarwoche hinein vorbereitet. Die Deiche sind massiv aufgeweicht, was große Sorge bereitet. Das Land Niedersachsen ist mit Rettungskräften gut aufgestellt und geht davon aus, die Lage auch über Silvester mit eigenen Kräften bewältigen zu können.', role='assistant'))], created=1703939215, model='together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=2376, completion_tokens=160, total_tokens=2536), _response_ms=3085.318)
Die Hochwasserlage in Niedersachsen ist weiter k

# Do Benchmark

In [41]:
import tiktoken
import textwrap
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import json
from io import StringIO

# load input file
workbook = load_workbook(filename = 'llm_benchmark.xlsx', data_only=True) # data_only=True
worksheet_emails = workbook['Emails'] # workbook.active
worksheet_llm_config = workbook['LLM-Configs']

# create log file (based on input file, sheet 'Results', all other sheets are deleted)
workbook_log = load_workbook(filename = 'llm_benchmark.xlsx')
workbook_log.remove(workbook_log['README'])
workbook_log.remove(workbook_log['Emails'])
workbook_log.remove(workbook_log['LLM-Configs'])
worksheet_log = workbook_log['Results']
while(worksheet_log.max_row > 1): # delete log rows: continuously delete row 2 until there is only first row (header row) left over
  worksheet_log.delete_rows(2)

# create headers dictionaries to access columns
column_email = {}
for column_index, cell in enumerate(worksheet_emails["1"]):
  column_email[cell.value] = column_index + 1
column_llm_config = {}
for column_index, cell in enumerate(worksheet_llm_config["1"]):
  column_llm_config[cell.value] = column_index + 1

# iterate through all lines
for row_index, row1 in enumerate(worksheet_emails):
  if row_index == 12: # > 0  //  == 12
    dateiname = worksheet_emails.cell(row=row_index+1, column=column_email["Dateiname"]).value
    if dateiname != None:
      ic(dateiname)
      prompt_context = worksheet_emails.cell(row=row_index+1, column=column_email["Prompt-Context"]).value
      target_JSON = worksheet_emails.cell(row=row_index+1, column=column_email["Ziel-JSON"]).value
      llm_configs = worksheet_emails.cell(row=row_index+1, column=column_email["LLM-Configs"]).value.split(',')

      for config_index, config in enumerate(llm_configs):
        # search config
        found_flag = False
        for llm_config_index, row2 in enumerate(worksheet_llm_config):
          llm_config_id = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["LLM-Config-ID"]).value
          if llm_config_id.strip() == config.strip():
            found_flag = True
            break
        if found_flag == True:
          ic('LLM-Config found:', dateiname, config)
          llm_config_LLM_Model = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["LLM-Model"]).value
          llm_config_Prompt = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Prompt"]).value
          llm_config_Response_Format = worksheet_llm_config.cell(row=llm_config_index+1, column=column_llm_config["Response-Format"]).value
          prompt_full = llm_config_Prompt + '\n\n' + prompt_context
          prompt_full_tokenlength = len(tiktoken.get_encoding('cl100k_base').encode(prompt_full))
          #for x in prompt_full[:500].split('\n'):
          #  print(textwrap.fill(x, 80))

          #
          # do LLM call here
          #
          start_time = time.time()
          response = completion(
            #model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
            model=llm_config_LLM_Model,
            messages=[{ "content": prompt_full,"role": "user"}],
            #temperature=0.5,
            max_tokens=500
          )
          response_txt = response.choices[0].message.content
          response_JSON = ''
          if llm_config_Response_Format == "JSON":
            response_JSON = response_txt
          else:
            if llm_config_Response_Format == "CSV":
              response_IO = StringIO(response_txt) # [4:] strip away first 4 chars "AI: "
              response_df = pd.read_csv(response_IO, sep="|")
              response_JSON_Obj = json.loads(response_df.to_json(orient='table',index=False))['data'][0]
              response_JSON = json.dumps(response_JSON_Obj) # ,indent=4

          response_model = response.model
          response_responsetime = float(response._response_ms)/1000
          time.sleep(1)
          #response = "response-obj"
          #response_txt = "dummy text"
          stop_time = time.time()
          elapsed_time = stop_time-start_time

          # log result
          current_date_time = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
          # Time	Email-Name
          log_row = [current_date_time, # Time
                     dateiname,         # Email-Name
                     config.strip(),    # LLM-Config-ID
                     llm_config_LLM_Model, # Input-Model
                     prompt_full,       # Input-Full-Prompt
                     prompt_context,    # Input-Context
                     str(response),     # Response-Full
                     response_model,    # Response-Model
                     response_responsetime, # Response-Duration
                     elapsed_time,      # Duration-end-to-end
                     response_txt,      # Ergebnis
                     response_JSON,     # Ergebnis-JSON
                     target_JSON         # Ziel-JSON

          ]
          worksheet_log.append(log_row)
        else:
          # LLM-Config not found - log result
          ic('LLM-Config not found:', dateiname, config)
          current_date_time = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
          log_row = [current_date_time, dateiname,'NOT FOUND: ' + config.strip()]
          worksheet_log.append(log_row)

# Save the log file with timestamp
current_date_time = '_' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
current_date_time = "" # for debugging
workbook_log.save('llm_benchmark_log' + current_date_time + '.xlsx')

ic| dateiname: '24optimized.msg'
ic| 'LLM-Config found:': 'LLM-Config found:'
    dateiname: '24optimized.msg'
    config: 'Simple_Prompt_1_GPT3_5'
ic| 'LLM-Config found:': 'LLM-Config found:'
    dateiname: '24optimized.msg'
    config: ' Simple_Prompt_1_GPT4'
ic| 'LLM-Config found:': 'LLM-Config found:'
    dateiname: '24optimized.msg'
    config: ' KOR_Prompt_1_GPT3_5'
ic| 'LLM-Config found:': 'LLM-Config found:'
    dateiname: '24optimized.msg'
    config: ' KOR_Prompt_2_GPT4'
ic| 'LLM-Config not found:': 'LLM-Config not found:'
    dateiname: '24optimized.msg'
    config: ' wrong_config '


In [48]:
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

# load log file
workbook_log = load_workbook(filename = 'llm_benchmark_log.xlsx') # data_only=True
worksheet_log = workbook_log['Results']

# create headers dictionaries to access columns
column_log = {}
for column_index, cell in enumerate(worksheet_log["1"]):
  column_log[cell.value] = column_index + 1
ic(column_index)

# iterate through all lines
for log_index, row3 in enumerate(worksheet_log):
  if log_index == 1: # > 0  //  == 1
    log_time = worksheet_log.cell(row=log_index+1, column=column_log["Time"]).value
    ic('found', log_time)

ic| column_index: 12
ic| 'found', log_time: '02.01.2024 15:35:52'
